In [1]:
import os

import pandas as pd
import numpy as np

import datetime as dt

import flask
from flask import Flask, request

import pickle

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from textblob import TextBlob
import string

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
with open('listing_frame_final','rb') as f:
    lff=pickle.load(f)
f.close()

In [8]:
lff

,beds,baths,price,sqft,stories,rooms,building_age,pol,sub,lda_topic0,...,Commercial,Condo,Multi-Family Home,Other,Single-Family Home,Bronx,Brooklyn,Manhattan,Queens,Staten Island
3,3.0,3.0,665000.0,1350.0,3.0,6.0,19.0,0.417532,0.530519,0.009693,...,0.0,0,0,0,1,0.0,0,0,0,1
8,1.0,1.0,459000.0,532.0,4.0,2.0,5.0,0.122857,0.495714,0.009472,...,0.0,1,0,0,0,0.0,1,0,0,0
15,5.0,5.0,1999999.0,3114.0,3.0,12.0,122.0,0.262141,0.528006,0.056832,...,0.0,1,0,0,0,0.0,1,0,0,0
18,6.0,5.0,1190000.0,3700.0,3.0,12.0,51.0,0.330556,0.701190,0.006829,...,0.0,0,1,0,0,0.0,1,0,0,0
20,2.0,1.0,849000.0,1200.0,6.0,5.0,104.0,0.303194,0.571307,0.066964,...,0.0,1,0,0,0,0.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2.0,3.0,475250.0,1500.0,2.0,3.0,34.0,0.344416,0.448052,0.005507,...,0.0,0,0,0,1,0.0,0,0,0,1
67,3.0,4.0,750000.0,1600.0,2.0,7.0,71.0,0.285415,0.506478,0.035465,...,0.0,0,0,1,0,0.0,1,0,0,0
68,5.0,4.0,280000.0,2389.0,2.0,13.0,101.0,0.232639,0.361042,0.026434,...,0.0,1,0,0,0,0.0,1,0,0,0
70,2.0,3.0,1490000.0,1222.0,6.0,3.0,12.0,0.206228,0.485877,0.020927,...,0.0,1,0,0,0,0.0,1,0,0,0


In [19]:
field='building_age'
print(lff[field].min(),lff[field].max())
lff.sort_values(by=field)

0.0 2020.0


,beds,baths,price,sqft,stories,rooms,building_age,pol,sub,lda_topic0,...,Commercial,Condo,Multi-Family Home,Other,Single-Family Home,Bronx,Brooklyn,Manhattan,Queens,Staten Island
4955,1.0,2.0,2725000.0,1254.0,21.0,3.0,0.0,0.124218,0.448633,0.071149,...,0.0,1,0,0,0,0.0,0,1,0,0
1799,5.0,5.0,1475000.0,4410.0,2.0,8.0,0.0,0.244781,0.548148,0.022248,...,0.0,0,0,0,1,0.0,0,0,0,1
4255,1.0,2.0,2500000.0,1098.0,21.0,4.0,0.0,0.127651,0.431468,0.072030,...,0.0,1,0,0,0,0.0,0,1,0,0
6697,1.0,2.0,2570000.0,1098.0,21.0,4.0,0.0,0.127651,0.431468,0.072030,...,0.0,1,0,0,0,0.0,0,1,0,0
1773,3.0,4.0,620000.0,1516.0,3.0,6.0,0.0,0.227273,0.424242,0.009032,...,0.0,1,0,0,0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7802,0.0,1.0,525000.0,658.0,9.0,2.0,185.0,0.265188,0.484060,0.027094,...,0.0,1,0,0,0,0.0,0,1,0,0
1104,3.0,2.0,415000.0,1956.0,3.0,7.0,186.0,0.274359,0.454103,0.013218,...,0.0,0,0,0,1,0.0,0,0,0,1
4366,3.0,3.0,4650000.0,2684.0,3.0,10.0,188.0,0.215904,0.440781,0.040312,...,0.0,1,0,0,0,0.0,0,1,0,0
3122,4.0,3.0,633000.0,1296.0,2.0,8.0,2020.0,0.411905,0.694048,0.007490,...,0.0,0,0,1,0,0.0,1,0,0,0


In [22]:
0 in range(50)

True